In [1]:
import os

In [2]:
pwd

'd:\\Chest-Cancer-classification\\expariment'

In [3]:
os.chdir('../')

In [4]:
pwd

'd:\\Chest-Cancer-classification'

In [54]:
from dataclasses import dataclass
from pathlib import Path

#define data path
@dataclass(frozen=True)
class DataIngestionConfig:
    dir :Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [55]:

from src.utils.main_utils import read_yaml,create_dir

In [56]:
from src.constant import *
import sys

In [57]:
from src.logging.logger import logging
from src.exception.exception import CustomException
import zipfile
import sys,os

In [58]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dir([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_dir([config.dir])

        data_ingestion_config = DataIngestionConfig(
            dir=config.dir,
           source_url=config.source_url,
           local_data_file  =config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [64]:
import urllib.request as request
import gdown

In [73]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig) -> None:
        
        self.config=config

    def download_file(self):
        '''
        fetch data from url
        '''
        try:
            dataset_url=self.config.source_url
            zip_download=self.config.local_data_file
            os.makedirs(self.config.dir,exist_ok=True)
            logging.info(f'Download da from {dataset_url} into {zip_download} ')

            
            prefix='https://drive.google.com/uc?/export=download&id='
            id=dataset_url.split('/')[5]
            gdown.download(prefix+id,zip_download)

            logging.info
        except Exception as e:
            logging.info('error occured ', e)
            raise CustomException(sys,e) 
        

    def extrat_zip_file(self):
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

        except Exception as e:
            logging.error(f'Error occurred: {e}')  
            raise CustomException(sys, e)

In [74]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extrat_zip_file()
except Exception as e:
    logging.info(f' Error in {str(e)}')
    raise CustomException(sys,e)
   

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1ZrFE6RSlNerUfwexpGlhc2r3haiV-CIz
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1ZrFE6RSlNerUfwexpGlhc2r3haiV-CIz&confirm=t&uuid=4532c522-5a7a-4232-9c86-925c3357d6ee
To: d:\Chest-Cancer-classification\artifacts\data_ingestion\data.zip
100%|██████████| 124M/124M [01:02<00:00, 1.98MB/s] 
